# Two-body - MC

In [ ]:
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir

%cd $workdir

In [ ]:
# Import packages

import os
import json
import numpy as np
import matplotlib as mpl, matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 100%; }
    div#menubar-container     { width: 100%; }
    div#maintoolbar-container { width: 100%; }
</style>
"""))

plt.rcParams.update({'font.size':16,'legend.frameon':True,'figure.figsize':[12,8],'xtick.major.size':7,'ytick.major.size':7,'legend.labelspacing':1})

# Solution conditions and parameters

In [ ]:
# Solution conditions

###########################################################################################
pHs = {'pH2': { 'pH': [2], 'N':7, 'input':workdir+'/../One_body/Charges_eq/Q_eq_pH2.pqr'},
       'pH5': { 'pH': [5], 'N':3, 'input':workdir+'/../One_body/Charges_eq/Q_eq_pH5.pqr'}}
###########################################################################################
Cp_range = [0.001, 0.002, 0.0040, 0.0070, 0.01, 0.015, 0.02, 0.025, 0.03]
#########################################################################

#LJ parameters                                                                                                                                       
eps = 0.12395 # epsilon for not hydrophobic in kJ/mol equal to 0.05 kT  
                                                                                                             
# Simulations steps                                                                                                         
micro = 1000
macro = 1100
nskip = 100

# Functions

In [ ]:
# Define the lenght of th cube as a function of the protein concentration

def lenght(Cp):
    Na = 6.022e23
    N = 2
    V = N *1e27/Na/Cp
    return np.cbrt(V)

In [ ]:
def mkinput():
    js = {"temperature": 298.15,
             "geometry": {"type":"cuboid", "length": l}, 
               "mcloop": {"macro": macro, "micro": micro},
               "random": {"seed": "hardware"},
               "energy": [{"nonbonded_coulomblj": 
                                 {"lennardjones": {"mixing": "LB", 
                                                   "custom": {"Cl HNTR": {"sigma": 4.70,"eps": 0.01},
                                                              "Cl HLYS": {"sigma": 4.70,"eps": 0.01},
                                                              "Cl HARG": {"sigma": 3.50,"eps": 0.01}
                                                              
                                                             }},
                                      "coulomb": {"type":"plain","epsr": 78.7}}}
                         ],

               "atomlist": [
                ##############
                #### Salt ####
                ##############
                #{"Na" : {"mw":22.99, "sigma":4.6, "eps": eps, "q": 1, "dp": 50}},
                {"Cl" : {"mw":35.45, "sigma":4.6, "eps": eps, "q": -1, "dp": 50 }},
                ###################
                #### TERMINALS ####
                ###################
                {"CTR": {"mw": 16.0,"sigma": 3.13,"eps": eps,"q": -1}},
                {"HCTR": {"mw": 16.0,"sigma": 3.13,"eps": eps,"q": 0}},
                {"NTR": {"mw": 14.0,"sigma": 2.99,"eps": eps,"q": 0}},
                {"HNTR": {"mw": 14.0,"sigma": 2.99,"eps": eps,"q": 1}},
                ###################
                ####Hydrophobic####
                ###################
                {"ALA": {"mw": 66.0,"sigma": 5.02,"eps": eps,"q": 0}},
                {"ILE": {"mw": 102.0,"sigma": 5.8,"eps": eps,"q": 0}},
                {"LEU": {"mw": 102.0,"sigma": 5.8,"eps": eps,"q": 0}},
                {"MET": {"mw": 122.0,"sigma": 6.16,"eps": eps,"q": 0}},
                {"PHE": {"mw": 138.0,"sigma": 6.41,"eps": eps,"q": 0}},
                {"VAL": {"mw": 90.0,"sigma": 5.56,"eps": eps,"q": 0}},
                {"PRO": {"mw": 90.0,"sigma": 5.56,"eps": eps,"q": 0}},
                {"GLY": {"mw": 54.0,"sigma": 4.69,"eps": eps,"q": 0}},
                #########################
                #### Not Hydrophobic ####
                #########################
                {"ARG": {"mw": 144.0,"sigma": 6.50,"eps": eps,"q": 0}},
                {"HARG": {"mw": 144.0,"sigma": 6.50,"eps": eps,"q": 1}},
                {"LYS": {"mw": 116.0,"sigma": 6.05,"eps": eps,"q": 0}},
                {"HLYS": {"mw": 116.0,"sigma": 6.05,"eps": eps,"q": 1}},
                {"ASP": {"mw": 110.0,"sigma": 5.94,"eps": eps,"q": -1}},
                {"HASP": {"mw": 110.0,"sigma": 5.94,"eps": eps,"q": 0}},
                {"LASP": {"mw": 110.0,"sigma": 5.94,"eps": eps,"q": 2}},
                {"GLU": {"mw": 122.0,"sigma": 6.16,"eps": eps,"q": -1}},
                {"HGLU": {"mw": 122.0,"sigma": 6.16,"eps": eps,"q": 0}},
                {"LGLU": {"mw": 122.0,"sigma": 6.16,"eps": eps,"q": 2}},
                {"GLN": {"mw": 120.0,"sigma": 6.12,"eps": eps,"q": 0}},
                {"ASN": {"mw": 108.0,"sigma": 5.91,"eps": eps,"q": 0}},
                {"HIS": {"mw": 130.0,"sigma": 6.29,"eps": eps,"q": 0}},
                {"HHIS": {"mw": 130.0,"sigma": 6.29,"eps": eps,"q": 1}},
                {"SER": {"mw": 82.0,"sigma": 5.39,"eps": eps,"q": 0}},
                {"THR": {"mw": 94.0,"sigma": 5.64,"eps": eps,"q": 0}},
                {"TYR": {"mw": 154.0, "sigma": 6.65,"eps": eps, "q": -1}},
                {"HTYR": {"mw": 154.0,"sigma": 6.65,"eps": eps,"q": 0}},
                {"CYS": {"mw": 103.0,"sigma": 5.72,"eps": eps,"q": -1}},
                {"HCYS":{"mw": 103.0,"sigma": 5.72,"eps": eps,"q": 0}},
                {"CYX": {"mw": 103.0,"sigma": 5.72,"eps": eps,"q": 0}},
                {"TRP": {"mw": 176.0,"sigma": 6.95,"eps": eps,"q": 0}},
                {"H": {"pactivity": pH, "implicit": True } }
                    ],
          
       
          "moleculelist": [{"protein": {"structure": Input, "insdir":[0,0,0], "insoffset":[0,0,0], "keepcharges":True, 'rotate': False, 'keeppos':False, 'rigid':False}},
                           {"CL": {"atoms": ["Cl"], "atomic":True}}
                          ],
          
          "insertmolecules": [{"protein": {"N": 2, "inactive": False }},
                              {"CL": {"N": 2*N, "inactive": False}}
                             ],

          
          "moves": 
                [{"moltransrot": {"molecule": "protein", "dir":[1,1,1], "dprot":1, "dp":20,  "repeat": 10}},
                    {"transrot": {"molecule": "CL", "dir":[1,1,1], "repeat":10}}
                 ],
                 

          "analysis": [
            {"density": { "nstep": 1000, 'nskip': nskip}}, # average concentration of molecules in moleculelist ,<N/V>, in out.json file (see also bulk density in Faunus manual fot rho-name.dat files)
            {"multipole": {"nstep": 1000, 'nskip': nskip} }, # average charge of protein in out.json file
            {"savestate": {"file": "state.json"} },  # to restart the simulation
            {"xtcfile": {"file": "traj.xtc", "nstep": 10000, 'nskip': nskip} },
            {"systemenergy": {"file": "energy.dat", "nstep": 1000, 'nskip': nskip} },
            {"virtualvolume": {"dV": 0.1, "nstep": 1, 'nskip': nskip}}
            ]}  

    with open('input.json', 'w') as f:
        json.dump(js, f, indent=4)



#############################################################################################

for name, prop in pHs.items():
    %mkdir $name
    %cd -q $name
    for pH in prop ['pH']:
        %mkdir $pH
        %cd -q $pH
        for Cp in Cp_range:
            
            %mkdir $Cp
            %cd -q $Cp
            
            l = lenght(Cp)
            N = prop['N']
            Input= prop['input'] 
            mkinput()

            #exists = os.path.isfile('state.json')
            #if exists:
                #!export OMP_NUM_THREADS=1;/Users/marco/faunus/faunus -i input.json -s state.json -v 6 #> out
            #    !sbatch ../../../submit_state.sh   
            #else:
                #!export OMP_NUM_THREADS=1;/Users/marco/faunus/faunus -i input.json  -v 6 #> out
            #    !sbatch ../../../submit.sh
                
            %cd -q ../
        %cd -q ../
    %cd -q ../